# Zenith JAX Core Integration - Phase 1 Validation

**Date:** 2025-12-28  
**Purpose:** Validate Phase 1 implementation with real JAX on GPU

---

In [ ]:
# STEP 1: Clean install from GitHub
# This ensures we get the LATEST version with all fixes
!pip uninstall pyzenith -y 2>/dev/null
!pip cache purge 2>/dev/null
!pip install --force-reinstall --no-cache-dir git+https://github.com/vibeswithkk/ZENITH.git -q

# Verify installation
import zenith
print(f"Zenith version: {zenith.__version__}")

# Check JAX
import jax
import jax.numpy as jnp
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

In [ ]:
# STEP 2: Import Phase 1 modules
from zenith.jax.checkpointing import (
    OptimalCheckpointSelector,
    CheckpointPolicy,
    checkpoint,
)
from zenith.jax.memory_manager import (
    JAXActivationStore,
    EvictionPolicy,
    compute_array_size,
    get_device_string,
)
from zenith.jax.mixed_precision import (
    MixedPrecisionPolicy,
    DynamicLossScaler,
    LossScalerConfig,
    ZenithMixedPrecision,
    create_policy,
    detect_best_precision,
)

print("All Phase 1 modules imported!")

---
## Section 1: Gradient Checkpointing

In [ ]:
# TEST 1.1: OptimalCheckpointSelector
print("="*60)
print("TEST 1.1: OptimalCheckpointSelector")
print("="*60)

for n in [4, 12, 24, 48]:
    sel = OptimalCheckpointSelector(num_layers=n)
    sqrt = sel.select_sqrt()
    dp = sel.select_dp()
    red = sel.estimate_memory_reduction(sqrt)
    print(f"Layers={n}: sqrt={len(sqrt)}, DP={len(dp)}, reduction={red:.1f}%")

print("\n[PASS] TEST 1.1")

In [ ]:
# TEST 1.2: JAX Checkpoint with Gradients
print("="*60)
print("TEST 1.2: JAX Checkpoint with Gradients")
print("="*60)

def mlp(x, w1, w2):
    return jnp.dot(jax.nn.relu(jnp.dot(x, w1)), w2)

ckpt_mlp = jax.checkpoint(mlp)

key = jax.random.PRNGKey(42)
x = jax.random.normal(key, (32, 64))
w1 = jax.random.normal(key, (64, 128))
w2 = jax.random.normal(key, (128, 64))

grads = jax.grad(lambda x,w1,w2: jnp.mean(ckpt_mlp(x,w1,w2)**2), argnums=(1,2))(x,w1,w2)
print(f"Grad shapes: {grads[0].shape}, {grads[1].shape}")
assert jnp.all(jnp.isfinite(grads[0])) and jnp.all(jnp.isfinite(grads[1]))

print("\n[PASS] TEST 1.2")

In [ ]:
# TEST 1.3: Zenith checkpoint() wrapper
print("="*60)
print("TEST 1.3: Zenith checkpoint() wrapper")
print("="*60)

def fn(x, w):
    return jax.nn.relu(jnp.dot(x, w))

ckpt_fn = checkpoint(fn, policy=CheckpointPolicy.DOTS_SAVEABLE)

key = jax.random.PRNGKey(0)
x = jax.random.normal(key, (8, 16))
w = jax.random.normal(key, (16, 16))

out = ckpt_fn(x, w)
grad_w = jax.grad(lambda w: jnp.mean(ckpt_fn(x, w)**2))(w)

print(f"Output: {out.shape}, Grad: {grad_w.shape}")
assert jnp.all(jnp.isfinite(grad_w))

print("\n[PASS] TEST 1.3")

---
## Section 2: Memory Management

In [ ]:
# TEST 2.1: JAXActivationStore
print("="*60)
print("TEST 2.1: JAXActivationStore")
print("="*60)

store = JAXActivationStore(max_memory_bytes=100*1024*1024)
key = jax.random.PRNGKey(0)
arrays = {}

for i in range(5):
    arr = jax.random.normal(key, (1024, 1024))
    arrays[i] = arr
    store.store(layer_id=i, array=arr)
    print(f"Stored layer {i}: {compute_array_size(arr)/1024/1024:.2f} MB")

for i in range(5):
    ret = store.retrieve(layer_id=i)
    assert ret is not None and jnp.allclose(ret, arrays[i])

print("\n[PASS] TEST 2.1")

In [ ]:
# TEST 2.2: Eviction Under Memory Pressure
print("="*60)
print("TEST 2.2: Eviction Under Memory Pressure")
print("="*60)

# 5 MB budget, 10 x 1MB arrays = must evict
store = JAXActivationStore(max_memory_bytes=5*1024*1024, eviction_policy=EvictionPolicy.LRU)
key = jax.random.PRNGKey(42)

for i in range(10):
    arr = jax.random.normal(key, (512, 512))  # 1MB each
    store.store(layer_id=i, array=arr)

stats = store.statistics
print(f"Stored: {stats['store_count']}, Evicted: {stats['eviction_count']}")
print(f"Current: {stats['current_memory_mb']:.2f} MB, In store: {stats['stored_count']}")

assert stats['eviction_count'] > 0, "Eviction should have occurred!"

print("\n[PASS] TEST 2.2")

In [ ]:
# TEST 2.3: Device Detection
print("="*60)
print("TEST 2.3: Device Detection")
print("="*60)

arr = jax.random.normal(jax.random.PRNGKey(0), (100, 100))
dev = get_device_string(arr)
print(f"Array device: {dev}")

print("\n[PASS] TEST 2.3")

---
## Section 3: Mixed Precision

In [ ]:
# TEST 3.1: MixedPrecisionPolicy
print("="*60)
print("TEST 3.1: MixedPrecisionPolicy")
print("="*60)

for mode in ['fp32', 'bf16', 'fp16']:
    p = create_policy(mode)
    print(f"{mode}: compute={p.compute_dtype}, scaling={p.requires_loss_scaling}")

arr = jnp.ones((10, 10), dtype=jnp.float32)
arr_bf16 = arr.astype(jnp.bfloat16)
assert arr_bf16.dtype == jnp.bfloat16

print("\n[PASS] TEST 3.1")

In [ ]:
# TEST 3.2: DynamicLossScaler
print("="*60)
print("TEST 3.2: DynamicLossScaler")
print("="*60)

scaler = DynamicLossScaler(LossScalerConfig(
    initial_scale=2**15,
    growth_factor=2.0,
    backoff_factor=0.5,
    growth_interval=5,
))

print(f"Initial scale: {scaler.scale}")

key = jax.random.PRNGKey(0)
params = jax.random.normal(key, (64, 64))
x = jax.random.normal(key, (32, 64))

def loss_fn(p, x):
    return jnp.mean((jnp.dot(x, p)) ** 2)

for step in range(10):
    def scaled_loss(p):
        return scaler.scale_loss(loss_fn(p, x))
    
    grads = jax.grad(scaled_loss)(params)
    unscaled, is_finite = scaler.unscale_grads({'p': grads})
    scaler.update(is_finite)
    
    if step % 3 == 0:
        print(f"Step {step}: scale={scaler.scale:.0f}, finite={is_finite}")

print("\n[PASS] TEST 3.2")

In [ ]:
# TEST 3.3: ZenithMixedPrecision
print("="*60)
print("TEST 3.3: ZenithMixedPrecision")
print("="*60)

mp = ZenithMixedPrecision(policy='bf16')
print(f"Policy: {mp.policy.mode.value}")

key = jax.random.PRNGKey(0)
params = {
    'w1': jax.random.normal(key, (64, 128), dtype=jnp.float32),
    'w2': jax.random.normal(key, (128, 64), dtype=jnp.float32),
}

compute_params = mp.cast_to_compute(params)
print(f"Original: {params['w1'].dtype} -> Compute: {compute_params['w1'].dtype}")
assert compute_params['w1'].dtype == jnp.bfloat16

back = mp.cast_to_param(compute_params)
print(f"Back: {back['w1'].dtype}")
assert back['w1'].dtype == jnp.float32

print("\n[PASS] TEST 3.3")

In [ ]:
# TEST 3.4: Hardware Detection
print("="*60)
print("TEST 3.4: Hardware Detection")
print("="*60)

best = detect_best_precision()
print(f"Best precision: {best}")
for d in jax.devices():
    print(f"  Device: {d}")

print("\n[PASS] TEST 3.4")

---
## Section 4: Benchmark

In [ ]:
# BENCHMARK: Mixed Precision Speedup
print("="*60)
print("BENCHMARK: Mixed Precision Speedup")
print("="*60)

import time

SIZE = 2048
key = jax.random.PRNGKey(0)
a32 = jax.random.normal(key, (SIZE, SIZE), dtype=jnp.float32)
b32 = jax.random.normal(key, (SIZE, SIZE), dtype=jnp.float32)
a16 = a32.astype(jnp.bfloat16)
b16 = b32.astype(jnp.bfloat16)

matmul = jax.jit(jnp.dot)

# Warmup
matmul(a32, b32).block_until_ready()
matmul(a16, b16).block_until_ready()

N = 20
t = time.time()
for _ in range(N): matmul(a32, b32).block_until_ready()
t32 = (time.time() - t) / N * 1000

t = time.time()
for _ in range(N): matmul(a16, b16).block_until_ready()
t16 = (time.time() - t) / N * 1000

print(f"FP32: {t32:.2f} ms")
print(f"BF16: {t16:.2f} ms (speedup: {t32/t16:.2f}x)")

print("\n[BENCHMARK COMPLETE]")

In [ ]:
# FINAL SUMMARY
print("="*70)
print("ZENITH JAX PHASE 1 VALIDATION - ALL TESTS PASSED!")
print("="*70)
print("""
 Section 1: Gradient Checkpointing
   [PASS] TEST 1.1: OptimalCheckpointSelector
   [PASS] TEST 1.2: JAX checkpoint with gradients
   [PASS] TEST 1.3: Zenith checkpoint() wrapper

 Section 2: Memory Management
   [PASS] TEST 2.1: JAXActivationStore
   [PASS] TEST 2.2: Eviction under pressure
   [PASS] TEST 2.3: Device detection

 Section 3: Mixed Precision
   [PASS] TEST 3.1: MixedPrecisionPolicy
   [PASS] TEST 3.2: DynamicLossScaler
   [PASS] TEST 3.3: ZenithMixedPrecision
   [PASS] TEST 3.4: Hardware detection

 Section 4: Performance
   [DONE] Mixed precision benchmark

Phase 1 JAX Core Integration VALIDATED!
Ready for Phase 2: XLA Backend & ONNX Export
""")